<a href="https://colab.research.google.com/github/onlyphantom/llm-python/blob/main/workshop/Generative_AI_Template_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generative AI for Financial Chatbots Development

Congratulations on making it this far in this self-paced Generative AI lesson series! Before you attempt this challenge, you should complete the workbook to have a baseline understanding of the materials presented in the challenge:

- [Generative AI Series 1: Generative AI for Finance](https://docs.sectors.app/recipes/generative-ai-python/01-background)
- [Generative AI Series 2: Tool Use and Function Calling for Finance LLMs](https://docs.sectors.app/recipes/generative-ai-python/02-tool-use)
- [Generative AI Series 3: Structured Output](https://docs.sectors.app/recipes/generative-ai-python/03-structured-output)
- [Generative AI Series 4: Conversational Tool Use AI](https://docs.sectors.app/recipes/generative-ai-python/04-conversational)

---

## Generative AI Workshop

The materials are specifically designed for the following workshop by Supertype, and it might be beneficial to join the workshop (\$9, +\$4 for certification grading, post-workshop support and API credits) for a live-instructor, hands-on experience if you're new to the topics covered.

- [Generative AI for financial chatbots workshop](https://supertype.ai/financial-chatbots/)

## Make a Copy for submission
Please use File > Save a Copy in Drive to duplicate this assignment template.

When you have completed the challenge, submit it to the GitHub discussion thread for grading! Good luck!

---

## Part 1: Text Extraction AI

For the Challenge in this chapter, we are going to build an AI agent that can (1) extract information from unstructured
text, (2) run validation checks on the extracted data based on schema constraints and business logic rules, and (3) generate a structured response ready
for downstream tools to process.

This has many practical applications. You can imagine an assistant chatbot that extract information from loose text such as news,
press releases, or even user's conversational queries, and then generate structured responses to be fed into a downstream tool. One might
also imagine a chatbot that allow user to upload a document, extract information, and then perform some actions based on the extracted data.

### 5 Instructions
There are 5 instructions in total. Each successful implementation earns you 1 point. Successfully running the following cell (`python -m pytest`) with the expectected output earns you another 1 point.

The total score for Part 1 is 6 points.

In [ ]:
!pip install langchain-core
!pip install langchain-openai
!pip install langgraph
!pip install langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.

In [ ]:
%%file test_parser.py

from typing import Optional
import pytest

from pydantic import BaseModel, Field, model_validator

from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate

from langchain_core.exceptions import OutputParserException

# 1. bring in your llm
# llm = ...


class Stock(BaseModel):
    """Information about a company's stock"""

    symbol: str = Field(description="The stock symbol")
    name: str = Field(
        description="The name of the company for which the stock symbol represents"
    )
    sector: Optional[str] = Field(default=None, description="The sector of the company")
    # 2. implement the other fields
    # ...

    @model_validator(mode="before")
    @classmethod
    def validate_symbol_4_letters(cls, values: dict) -> dict:
        # 3. implement LLM validation logic
        # ...
        pass

parser = PydanticOutputParser(pydantic_object=Stock)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

runnable = prompt | llm | parser


class TestParser:
    def test_output_parser_symbol_valid(self):
        text = """
        Bank Central Asia (BBCA) is a bank in Indonesia and is part of the finance sector.
            It is in the banking industry and has a market capitalization of $8.5 billion.
        """
        # 4. implement when symbol and market cap (and other fields) are all valid
        ...


    def test_output_parser_symbol_invalid(self):
        text = """
        Bank Central Asia (BCA) is a bank in Indonesia and is part of the finance sector.
            It is in the banking industry and has a market capitalization of $8.5 billion.
        """

        # assert exception is raised when the symbol is not 4 letters long
        with pytest.raises(OutputParserException):
            out = runnable.invoke(text)

    def test_output_parser_mcap_invalid(self):
        text = """
        Bank Central Asia (BBCA) is a bank in Indonesia and is part of the finance sector.
            It is in the banking industry and has a market capitalization of $-8.5 billion.
        """

        # 5. assert exception is raised when extraction task fail by detecting <0 market cap
        # ...



Overwriting test_parser.py


In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# 6. run this with 3 passes
!python -m pytest test_parser.py

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.5.0
rootdir: /content
plugins: anyio-3.7.1, typeguard-4.3.0
collected 3 items                                                                                  

test_parser.py ...                                                                           [100%]

======================================== 3 passed in 4.90s =========================================


- Do not alter any of the `text` prompt. Doing so invalidatest the purpose of the quiz / challenge.
- Each correct implementation gets you 1 point. Successfully executing the cell above (`python -m pytest test_parser.py`) with the expected output gets you another 1 point. You get a total of 5+1 points from this section above.  

## Part 2: A LangGraph ReAct Agent with retriever tools

In [ ]:
import json
import requests
from typing import List

from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage


SECTORS_API_KEY = userdata.get('SECTORS_API_KEY')
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

def retrieve_from_endpoint(url: str) -> dict:
    headers = {"Authorization": SECTORS_API_KEY}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.HTTPError as err:
        raise SystemExit(err)
    return json.dumps(data)


@tool
def get_company_overview(stock: str) -> str:
    """
    Get company overview

    @param stock: The stock symbol of the company
    @return: The company overview
    """

    url = f"https://api.sectors.app/v1/company/report/{stock}/?sections=overview"
    return retrieve_from_endpoint(url)

@tool
def get_top_companies_ranked(dimension: str) -> List[str]:
   # 7. implement this tool correctly, using the tool implementation above as reference
   pass


llm = ChatGroq(
    temperature=0,
    model_name="llama3-groq-70b-8192-tool-use-preview",
    groq_api_key=GROQ_API_KEY,
)

tools = [
    get_company_overview,
    get_top_companies_ranked,
]

# 8: ask that floating numbers are returned in 2 decimal points so the result is prettier
# return full company name, symbol, and the value (in the case of companies by p/e values, return the p/e
# but in 2 decimal points)
system_message = ""


# 9: implement the below correctly, with llm, tools, and system_message as state modifier
app = create_react_agent()

def query_app(text: str) -> str:
    out = app.invoke(
        {
            "messages": [
                HumanMessage(text),
            ]
        }
    )
    # return out["messages"][-1].content
    return out["messages"]

out_agent = query_app(
    "Get me the top 7 companies based on P/E values, along with their full company name and PE values"
)

print(out_agent[-1].content)


In [ ]:
# 10: follow up now with a second question, to get the overview of whichever symbol
# is 4th on the list above in `out_agent`

out_agent2 = ...

print(out_agent2[-1].content)

## Conclusion

Congratulations on making your way through the challenges. My hope is that you find the session educational and fun, and I have, in my own way, inspired you to dive deeper into the exciting world of building financial chat agents using information retriever tools!

Please submit your work at the GitHub repository discussion thread for grading. If you score 8/10 you will obtain a certification jointly issued by Supertype and Sectors.

If you need help, please reach out to us on Discord (exclusively for Practicum members).

Thank you again for your participation, and I hope you walked away with lots of new ideas on what to build next!